In [14]:
import pandas as pd
import numpy as np
import fatpack

In [15]:
def import_fastrun(filename,elimt=None):
    """ Import a .out fast output file in a pandas dataframe.
    
    INPUTS:
    ---------
    filename : path and filename ex. C:/example/fast.out

    elimt : flag for eliminating the first X seconds of a simulation

    OUTPUTS:
    ----------
    df : pandas dataframe of .out fast output file """

    # import file into pandas dataframe
    dfraw = pd.read_csv(filename, sep='\t',skiprows=5)
    # format dataframe
    dfraw = dfraw.drop([0]).reset_index(drop=True) # drop units
    dfraw.columns = dfraw.columns.str.replace(' ','') # drop whitespaces in names
    dfraw = dfraw.astype('float') # convert scientific numbers to floats

    # make new dataframe & eliminate transient effects
    if elimt==None: df = dfraw 
    else: df = dfraw[elimt:].reset_index(drop=True)

    return df

In [16]:
def get_DEL(fast_data, chan_dict, binNum=100, t=600):
        """ Calculates the short-term damage equivalent load of multiple variables
        
        Parameters: 
        -----------
        fast_data: list
            List of dictionaries containing openfast output data (returned from ROSCO_toolbox.FAST_IO.load_output)
        
        chan_dict : list, tuple
            tuple/list containing channel names to be analyzed and corresponding fatigue slope factor "m"
            ie. ('TwrBsFxt',4)
        
        binNum : int
            number of bins for rainflow counting method (minimum=100)
        
        t : float/int
            Used to control DEL frequency. Default for 1Hz is 600 seconds for 10min data
        
        Outputs:
        -----------
        dfDEL : pd.DataFrame
            Damage equivalent load of each specified variable for one fast output file  
        """
        # check data types
        assert isinstance(fast_data, (dict,list)), 'fast_data must be of type list'
        assert isinstance(chan_dict, (list,tuple)), 'chan_dict must be of type list or tuple'
        assert isinstance(binNum, (float,int)), 'binNum must be of type float or int'
        assert isinstance(t, (float,int)), 't must be of type float or int'

        # create dictionary from chan_dict
        dic = dict(chan_dict)

        # pre-allocate list
        dflist = []

        for fd in fast_data:
            # if self.verbose:
            #     print('Processing data for {}'.format(fd['meta']['name']))
            dlist = [] # initiate blank list every loop
            # loop through channels and apply corresponding fatigue slope
            for var in dic.keys():
                # find rainflow ranges
                ranges = fatpack.find_rainflow_ranges(np.asarray(fd[var]))

                # find range count and bin
                Nrf, Srf = fatpack.find_range_count(ranges,binNum)

                # get DEL
                DELs = Srf**dic[var] * Nrf / t
                DEL = DELs.sum() ** (1/dic[var])
                dlist.append(DEL)
            # append DEL values for each channel to master list
            dflist.append(dlist)
            
            # create dataframe to return
            dfDEL = pd.DataFrame(np.transpose(dflist))
            dfDEL = dfDEL.T
            dfDEL.columns = dic.keys()

        return dfDEL

In [21]:
df = import_fastrun('../test_data/Hitachi_023.out',elimt=100)
data = [df.to_dict('list'),df.to_dict('list')]

In [22]:
chandic = [
    ('TwrBsMyt',3),
    ('TwrBsMxt',3),
    ('RootMyb1',10)
]

In [23]:
dfDEL = get_DEL(data,chandic)